In [1]:
import pandas as pd
from hmmlearn import hmm
import numpy as np
from sklearn import preprocessing
from scipy.stats import norm

# Getting data and diving them into unique unit numbers

We need to divide data into unique numbers, because the state restes as the unit number changes, so we need to find Gaussian distribution for different unit numbers

In [2]:
data = pd.read_csv('/home/yannik/Documents/hitachi/CMAPSS/train_FD001.txt', sep=" ", header=None)
unique_unit_values = data[0].unique() #Number of units
data_cycles = []
for unit_num in unique_unit_values:
    data_cycles.append(data[data[0] == unit_num])

# Removing operational settings and normalize the data column wise

In [3]:
def normalize(data):
    x = data.values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    dataNew = pd.DataFrame(x_scaled)
    return dataNew
#Remove the operation settings
dataT = data[data.columns[5:26]]
dataT.columns = range(21)
dataT = normalize(dataT)

# Dividing data for each unit

I think this is why my transitional matrix previously was not working properly as in each unit the state resets and start from good condition

In [4]:
dataT_cycles = []
for unit_num in unique_unit_values:
    dataT_cycles.append(dataT[data[0] == unit_num])

# Identifying and removing non variable data columns

Removing the columns where the data does not vary

In [5]:
for dataT_cycle in dataT_cycles:
    print(dataT_cycle.columns[dataT_cycle.std() == 0])
"""
Here we can see 0,4,9,15,17,18 but also 5 at many places so we drop column number 5 as well
"""
dataT.drop(data.columns[[0, 3, 4, 5, 9, 15, 17, 18]],axis=1,inplace=True)
dataT.columns = range(13)
dataT_cycles = []
for unit_num in unique_unit_values:
    dataT_cycles.append(dataT[data[0] == unit_num])

Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='i

In [7]:
dataT_cycle[2]

20431    0.556573
20432    0.400916
20433    0.381731
20434    0.372575
20435    0.413778
           ...   
20626    0.587312
20627    0.729453
20628    0.684979
20629    0.746021
20630    0.639634
Name: 2, Length: 200, dtype: float64

# Using HMM to find out transitional matrices

Here we first define transmatrix as [[0.5, 0.5, 0.0, 0.0],[0.0, 0.5, 0.5, 0.0],[0.0, 0.0, 0.5, 0.5],[0.0,0.0,0.0,1.0]] which means there is half chance for each state to go to next state and half to remain in the current state itself.

Then we train for each unit for transmatrix as well as state means and we will take average of each unit transmatrices and states as the transmatrix and state

*Note*: Here state '0' means the perfect health and '3' means weakest health 

In [42]:
lr = hmm.GaussianHMM(n_components=4, covariance_type="diag",init_params="cm", params="mt")
lr.startprob_ = np.array([1.0, 0.0, 0.0, 0.0])
transmats = []
statemeans = []
for i in range(100):
    lr.transmat_ = np.array([[0.5, 0.5, 0.0, 0.0],[0.0, 0.5, 0.5, 0.0],[0.0, 0.0, 0.5, 0.5],[0.0,0.0,0.0,1.0]])
    lr.fit(dataT_cycles[i])
    transmat = lr.transmat_
    transmats.append(transmat)
    statemeans.append(lr.means_)

In [ ]:
dataT_cycles[]

In [43]:
transmat = np.array(transmats).mean(axis=0)
statemean = np.array(statemeans).mean(axis=0)

In [44]:
transmat

array([[0.62209581, 0.37790419, 0.        , 0.        ],
       [0.        , 0.71613744, 0.28386256, 0.        ],
       [0.        , 0.        , 0.87147068, 0.12852932],
       [0.        , 0.        , 0.        , 1.        ]])

In [45]:
pd.DataFrame(statemean)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.364569,0.354798,0.644379,0.249564,0.160976,0.318859,0.670973,0.270028,0.200299,0.376568,0.368885,0.595668,0.629249
1,0.444539,0.438584,0.565297,0.296442,0.196927,0.408062,0.577691,0.318774,0.226379,0.453613,0.441180,0.530597,0.536744
2,0.549558,0.509217,0.458618,0.365915,0.249928,0.539378,0.459054,0.383014,0.269767,0.558784,0.518762,0.425622,0.435775
3,0.683775,0.634612,0.325111,0.471230,0.316153,0.699910,0.305831,0.487424,0.325637,0.697557,0.643659,0.294516,0.294737


In [46]:
t_prob = np.array([transmat, transmat])

In [47]:
rewards = np.array([[100, 50, 0, -50],[-50, 0, 50, 100]])

In [48]:
e_prob = np.array([norm.pdf(statemean), norm.pdf(statemean)])

In [49]:
e_prob

array([[[0.3732922 , 0.37460638, 0.32414938, 0.3867102 , 0.3938067 ,
         0.37916873, 0.31852928, 0.38465978, 0.39101932, 0.37163604,
         0.37270181, 0.33408863, 0.32728779],
        [0.36140864, 0.36236026, 0.34003094, 0.38179271, 0.39128126,
         0.36707247, 0.33763089, 0.37917896, 0.38884972, 0.35993893,
         0.36194661, 0.34655797, 0.34542292],
        [0.34302724, 0.35043167, 0.35911824, 0.37310873, 0.38667503,
         0.34493384, 0.35904634, 0.37072733, 0.38468686, 0.34127789,
         0.34871667, 0.36439551, 0.36280549],
        [0.31577901, 0.32618028, 0.37840612, 0.35701865, 0.37949463,
         0.31227354, 0.38071473, 0.35425801, 0.37834145, 0.3127875 ,
         0.32429976, 0.38201006, 0.38198522]],

       [[0.3732922 , 0.37460638, 0.32414938, 0.3867102 , 0.3938067 ,
         0.37916873, 0.31852928, 0.38465978, 0.39101932, 0.37163604,
         0.37270181, 0.33408863, 0.32728779],
        [0.36140864, 0.36236026, 0.34003094, 0.38179271, 0.39128126,
         

In [50]:
# 0: no-repair, 1: repair
actions = ('0', '1')
# 0: failing, 1: low health, 2: good health, 3: perfect health
states = ('0', '1', '2', '3')

gamma = 0.95

In [51]:
"""
First we define an MDP. We also represent a policy
as a dictionary of {state: action} pairs, and a utility function as a
dictionary of {state: number} pairs. We then define the value_iteration
and policy_iteration algorithms."""


import random
import numpy as np
from collections import defaultdict

class MDP:

    """A Markov Decision Process, defined by an initial state, transition model,
    and reward function. We also keep track of a gamma value, for use by
    algorithms. The transition model is represented somewhat differently from
    the text. Instead of P(s' | s, a) being a probability number for each
    state/state/action triplet, we instead have T(s, a) return a
    list of (p, s') pairs. We also keep track of the possible states,
    terminal states, and actions for each state."""

    def __init__(self, init, actlist, terminals, transitions=None, reward=None, states=None, gamma=0.9):
        if not (0 < gamma <= 1):
            raise ValueError("An MDP must have 0 < gamma <= 1")

        # collect states from transitions table if not passed.
        self.states = states or self.get_states_from_transitions(transitions)
            
        self.init = init
        
        if isinstance(actlist, list):
            # if actlist is a list, all states have the same actions
            self.actlist = actlist

        elif isinstance(actlist, dict):
            # if actlist is a dict, different actions for each state
            self.actlist = actlist
        
        self.terminals = terminals
        self.transitions = transitions or {}
        if not self.transitions:
            print("Warning: Transition table is empty.")

        self.gamma = gamma

        self.reward = reward or {s: 0 for s in self.states}

        # self.check_consistency()

    def R(self, state):
        """Return a numeric reward for this state."""

        return self.reward[state]

    def T(self, state, action):
        """Transition model. From a state and an action, return a list
        of (probability, result-state) pairs."""

        if not self.transitions:
            raise ValueError("Transition model is missing")
        else:
            return self.transitions[state][action]

    def actions(self, state):
        """Return a list of actions that can be performed in this state. By default, a
        fixed list of actions, except for terminal states. Override this
        method if you need to specialize by state."""

        if state in self.terminals:
            return [None]
        else:
            return self.actlist

    def get_states_from_transitions(self, transitions):
        if isinstance(transitions, dict):
            s1 = set(transitions.keys())
            s2 = set(tr[1] for actions in transitions.values()
                     for effects in actions.values()
                     for tr in effects)
            return s1.union(s2)
        else:
            print('Could not retrieve states from transitions')
            return None

    def check_consistency(self):

        # check that all states in transitions are valid
        assert set(self.states) == self.get_states_from_transitions(self.transitions)

        # check that init is a valid state
        assert self.init in self.states

        # check reward for each state
        assert set(self.reward.keys()) == set(self.states)

        # check that all terminals are valid states
        assert all(t in self.states for t in self.terminals)

        # check that probability distributions for all actions sum to 1
        for s1, actions in self.transitions.items():
            for a in actions.keys():
                s = 0
                for o in actions[a]:
                    s += o[0]
                assert abs(s - 1) < 0.001

class POMDP(MDP):

    """A Partially Observable Markov Decision Process, defined by
    a transition model P(s'|s,a), actions A(s), a reward function R(s),
    and a sensor model P(e|s). We also keep track of a gamma value,
    for use by algorithms. The transition and the sensor models
    are defined as matrices. We also keep track of the possible states
    and actions for each state."""

    def __init__(self, actions, transitions=None, evidences=None, rewards=None, states=None, gamma=0.95):
        """Initialize variables of the pomdp"""

        if not (0 < gamma <= 1):
            raise ValueError('A POMDP must have 0 < gamma <= 1')

        self.states = states
        self.actions = actions

        # transition model cannot be undefined
        self.t_prob = transitions
        if not self.t_prob.any():
            print('Warning: Transition model is undefined')
        
        # sensor model cannot be undefined
        self.e_prob = evidences
        if not self.e_prob.any():
            print('Warning: Sensor model is undefined')
        
        self.gamma = gamma
        self.rewards = rewards

    def remove_dominated_plans(self, input_values):
        """
        Remove dominated plans.
        This method finds all the lines contributing to the
        upper surface and removes those which don't.
        """

        values = [val for action in input_values for val in input_values[action]]
        values.sort(key=lambda x: x[0], reverse=True)

        best = [values[0]]
        y1_max = max(val[1] for val in values)
        tgt = values[0]
        prev_b = 0
        prev_ix = 0
        while tgt[1] != y1_max:
            min_b = 1
            min_ix = 0
            for i in range(prev_ix + 1, len(values)):
                if values[i][0] - tgt[0] + tgt[1] - values[i][1] != 0:
                    trans_b = (values[i][0] - tgt[0]) / (values[i][0] - tgt[0] + tgt[1] - values[i][1])
                    if 0 <= trans_b <= 1 and trans_b > prev_b and trans_b < min_b:
                        min_b = trans_b
                        min_ix = i
            prev_b = min_b
            prev_ix = min_ix
            tgt = values[min_ix]
            best.append(tgt)

        return self.generate_mapping(best, input_values)

    def remove_dominated_plans_fast(self, input_values):
        """
        Remove dominated plans using approximations.
        Resamples the upper boundary at intervals of 100 and
        finds the maximum values at these points.
        """

        values = [val for action in input_values for val in input_values[action]]
        values.sort(key=lambda x: x[0], reverse=True)

        best = []
        sr = 100
        for i in range(sr + 1):
            x = i / float(sr)
            maximum = (values[0][1] - values[0][0]) * x + values[0][0]
            tgt = values[0]
            for value in values:
                val = (value[1] - value[0]) * x + value[0]
                if val > maximum:
                    maximum = val
                    tgt = value

            if all(any(tgt != v) for v in best):
                best.append(np.array(tgt))

        return self.generate_mapping(best, input_values)

    def generate_mapping(self, best, input_values):
        """Generate mappings after removing dominated plans"""

        mapping = defaultdict(list)
        for value in best:
            for action in input_values:
                if any(all(value == v) for v in input_values[action]):
                    mapping[action].append(value)

        return mapping

    def max_difference(self, U1, U2):
        """Find maximum difference between two utility mappings"""

        for k, v in U1.items():
            sum1 = 0
            for element in U1[k]:
                sum1 += sum(element)
            sum2 = 0
            for element in U2[k]:
                sum2 += sum(element)
        return abs(sum1 - sum2)

        
class Matrix:
    """Matrix operations class"""

    @staticmethod
    def add(A, B):
        """Add two matrices A and B"""

        res = []
        for i in range(len(A)):
            row = []
            for j in range(len(A[0])):
                row.append(A[i][j] + B[i][j])
            res.append(row)
        return res

    @staticmethod
    def scalar_multiply(a, B):
        """Multiply scalar a to matrix B"""

        for i in range(len(B)):
            for j in range(len(B[0])):
                B[i][j] = a * B[i][j]
        return B

    @staticmethod
    def multiply(A, B):
        """Multiply two matrices A and B element-wise"""

        matrix = []
        for i in range(len(B)):
            row = []
            for j in range(len(B[0])):
                row.append(B[i][j] * A[j][i])
            matrix.append(row)

        return matrix

    @staticmethod
    def matmul(A, B):
        """Inner-product of two matrices"""

        return [[sum(ele_a*ele_b for ele_a, ele_b in zip(row_a, col_b)) for col_b in list(zip(*B))] for row_a in A]

    @staticmethod
    def transpose(A):
        """Transpose a matrix"""
        
        return [list(i) for i in zip(*A)]


def pomdp_value_iteration(pomdp, epsilon=0.1):
    """Solving a POMDP by value iteration."""

    U = {'':[[0]* len(pomdp.states)]}
    count = 0
    while True:
        count += 1
        prev_U = U
        values = [val for action in U for val in U[action]]
        value_matxs = []
        for i in values:
            for j in values:
                value_matxs.append([i, j])

        U1 = defaultdict(list)
        for action in pomdp.actions:
            for u in value_matxs:
                u1 = Matrix.matmul(Matrix.matmul(pomdp.t_prob[int(action)], Matrix.multiply(pomdp.e_prob[int(action)], Matrix.transpose(u))), [[1], [1]])
                u1 = Matrix.add(Matrix.scalar_multiply(pomdp.gamma, Matrix.transpose(u1)), [pomdp.rewards[int(action)]])
                U1[action].append(u1[0])

        U = pomdp.remove_dominated_plans_fast(U1)
        # replace with U = pomdp.remove_dominated_plans(U1) for accurate calculations
        
        if count > 10:
            if pomdp.max_difference(U, prev_U) < epsilon * (1 - pomdp.gamma) / pomdp.gamma:
                return U



In [52]:
states

('0', '1', '2', '3')

In [53]:
pomdp = POMDP(actions, t_prob, e_prob, rewards, states, gamma)

In [54]:
utility = pomdp_value_iteration(pomdp, epsilon=3)
utility

defaultdict(list,
            {'0': [array([ 217.21201857,   86.47980271,  -38.52097818, -185.05578633])]})

# ADQRN

In [55]:
from __future__ import division

import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [56]:
#These lines establish the feed-forward part of the network used to choose actions
inputs1 = tf.placeholder(shape=[1,13],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([13,2],0,0.01))
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)

#Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
nextQ = tf.placeholder(shape=[1,2],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

In [107]:
def get_state(obs):
    state = 0
    diff = 16
    for i in range(len(statemean)):
        stateDiff = obs - statemean[i]
        stateDiffVal = np.sqrt(np.mean(stateDiff**2))
        if stateDiffVal < diff:
            diff = stateDiffVal
            state = i
    return state

In [130]:
# Getting the next step after an action is done

def getStepDetails(i,j,action):
    unitData = dataT_cycles[i]
    d = False
    if action == 1:
        newJ = 0
    else:
        newJ = j+1
    obsNext = unitData.values[newJ]
    if newJ >= len(unitData) - 1:
        d = True
    s1 = get_state(obsNext)
    r1 = rewards[action][s1]
    return r1,newJ,s1,obsNext,d

In [196]:
# Set learning parameters
init = tf.global_variables_initializer()
y = gamma
e = 0.1
num_episodes = len(dataT_cycles)
#create lists to contain total rewards and steps per episode
jList = []
rList = []
D = np.empty([0,5]) # Replay memory
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        #Reset environment and get first new observation for new unit
        rAll = 0
        d = False
        j = 0
        k = 0
        unitData = dataT_cycles[i]
        #The Q-Network
        while j < len(unitData):
            #Choose an action by greedily (with e chance of random action) from the Q-network
            a,allQ = sess.run([predict,Qout],feed_dict={inputs1:unitData.values[j].reshape(1,13)})
            if np.random.rand(1) < e:
                a[0] = np.random.randint(0,2)
            #Get new state and reward from environment
            r,j,s1,o1,d = getStepDetails(i,j,a[0])
            D = np.vstack([D, [a[0],unitData.values[j-1].reshape(1,13),r,o1,s1]])
            if len(D) > 20:
                lastInd = np.random.randint(15,len(D))
                randomSample = D[lastInd-15:lastInd]
                finalO = D[lastInd,3].reshape(1,13)
                Reward = np.sum(D[lastInd-15:lastInd,2])
            else:
                finalO = o1.reshape(1,13)
                Reward = r
            # We take batch size of 15 (j in algorithm)
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Qout,feed_dict={inputs1:finalO})
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = Reward + y*maxQ1
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:unitData.values[j-1].reshape(1,13),nextQ:targetQ})
            rAll += r
            s = s1
            k += 1
            if d == True or k >= 1000:
                #Reduce chance of random action as we train the model.
                e = 1./((i/50) + 10)
                break
        jList.append(j)
        rList.append(rAll)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


# Prediction

In [197]:
a = dataT_cycles[5].values[160].reshape(-1,13)

In [198]:
W1

array([[ 2127.6558 ,  6986.8765 ],
       [ 2248.1982 ,  5232.41   ],
       [ 3317.4592 ,  3077.835  ],
       [ 4824.3174 ,   -45.76535],
       [ 3286.811  ,  2219.9832 ],
       [ 2962.1672 ,  4778.8364 ],
       [ 5146.6387 ,  1414.4443 ],
       [ 4606.688  , -1017.36865],
       [ 8425.082  ,  1503.2651 ],
       [ 2555.8647 ,  2504.4    ],
       [ 1596.5271 ,  3793.9429 ],
       [ 3868.3757 ,   737.1053 ],
       [ 3159.9434 ,  1868.2078 ]], dtype=float32)

In [199]:
np.dot(a,W1)

array([[17942.35635803, 16792.32532451]])